In [2]:
import pdfplumber
import torch
from transformers import AutoTokenizer, AutoModel
import pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from tqdm import tqdm

In [4]:
# Step 1: Extract text from PDF
def extract_text_from_pdf(pdf_path):
    """
    Extracts text from each page of the provided PDF file.
    
    Args:
        pdf_path (str): The path to the PDF file.
        
    Returns:
        str: A string containing the extracted text from the PDF.
    """
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            text += page.extract_text() + "\n"  # Extract text only
    return text.strip()

# Usage
text = extract_text_from_pdf("book.pdf")
import re

In [5]:
def clean_extracted_text(text):
    """
    Cleans the extracted text by removing extra spaces and unwanted newlines.
    
    Args:
        text (str): The raw text extracted from the PDF.
        
    Returns:
        str: The cleaned text.
    """
    # Remove multiple spaces and replace with a single space
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Remove unwanted newlines but keep structured ones
    text = re.sub(r'\n+', '\n', text).strip()
    
    return text

In [6]:
# Clean the extracted text
text = clean_extracted_text(text)

In [7]:
text

'Textbook of Primary Care Dermatology David Buckley Paola Pasquali Editors 123 Textbook of Primary Care Dermatology David Buckley • Paola Pasquali Editors Textbook of Primary Care Dermatology Editors David Buckley Paola Pasquali The Ashe Street Clinic Department of Dermatology Tralee Pius Hospital de Valls Co. Kerry Valls Ireland Tarragona Spain ISBN 978-3-030-29100-6 ISBN 978-3-030-29101-3 (eBook) https://doi.org/10.1007/978-3-030-29101-3 © Springer Nature Switzerland AG 2021 This work is subject to copyright. All rights are reserved by the Publisher, whether the whole or part of the material is concerned, specifically the rights of translation, reprinting, reuse of illustrations, recitation, broadcasting, reproduction on microfilms or in any other physical way, and transmission or information storage and retrieval, electronic adaptation, computer software, or by similar or dissimilar methodology now known or hereafter developed. The use of general descriptive names, registered names,

In [12]:
def chunk_text(text, chunk_size=700, chunk_overlap=50):
    """
    Splits the cleaned text into smaller chunks for processing.
    
    Args:
        text (str): The cleaned text to be split.
        chunk_size (int): The maximum size of each text chunk.
        chunk_overlap (int): The number of overlapping characters between chunks.
        
    Returns:
        list: A list of text chunks.
    """
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    return text_splitter.split_text(text)

In [13]:
text_chunks = chunk_text(text)

In [14]:
def get_biobert_embeddings(text):
    """
    Generates embeddings for the given text using the BioBERT model.
    
    Args:
        text (str): The input text to generate embeddings for.
        
    Returns:
        numpy.ndarray: The generated embeddings as a NumPy array.
    """
    # Load BioBERT model and tokenizer
    tokenizer = AutoTokenizer.from_pretrained("monologg/biobert_v1.1_pubmed")
    model = AutoModel.from_pretrained("monologg/biobert_v1.1_pubmed")

    inputs = tokenizer(
        text,
        padding=True,
        truncation=True,
        return_tensors="pt",
        max_length=512
    )
    
    with torch.no_grad():
        outputs = model(**inputs)
    
    # Use mean pooling for sentence embedding
    embeddings = torch.mean(outputs.last_hidden_state, dim=1)
    return embeddings.numpy()

In [16]:

import os
# Initialize Pinecone client
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"))

index_name = "sknai"

# Check if index exists and create it if not
if index_name not in pinecone.list_indexes():
    pinecone.create_index(
        name=index_name,
        dimension=1536,  # Embedding dimension
        metric="cosine",  # Similarity metric
        cloud="aws",
        region="us-east-1"
    )

# Connect to the index
index = pinecone.Index(index_name)

AttributeError: init is no longer a top-level attribute of the pinecone package.

Please create an instance of the Pinecone class instead.

Example:

    import os
    from pinecone import Pinecone, ServerlessSpec

    pc = Pinecone(
        api_key=os.environ.get("PINECONE_API_KEY")
    )

    # Now do stuff
    if 'my_index' not in pc.list_indexes().names():
        pc.create_index(
            name='my_index', 
            dimension=1536, 
            metric='euclidean',
            spec=ServerlessSpec(
                cloud='aws',
                region='us-west-2'
            )
        )



In [ ]:
def upload_to_pinecone(chunks, batch_size=32):
    """
    Uploads text chunks and their embeddings to Pinecone in batches.
    
    Args:
        chunks (list): A list of text chunks to upload.
        batch_size (int): The number of chunks to upload in each batch.
    """
    batch = []
    for i, chunk in tqdm(enumerate(chunks), desc="Uploading to Pinecone"):
        # Generate embedding
        embedding = get_biobert_embeddings(chunk).tolist()[0]
        
        # Create metadata
        metadata = {
            "text": chunk,
            "chunk_id": str(i)
        }
        
        # Add to batch
        batch.append((str(i), embedding, metadata))
        
        # Upload in batches
        if len(batch) >= batch_size:
            index.upsert(vectors=batch)
            batch = []
    
    # Upload any remaining items
    if len(batch) > 0:
        index.upsert(vectors=batch)

# Usage
upload_to_pinecone(text_chunks)

1406it [05:26,  4.31it/s]
